In [1]:
import pandas as pd
import numpy as np 
from datetime import datetime,timezone
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import zstandard
import os
import sys
import logging.handlers
import re
from tqdm import tqdm
import csv

In [2]:
anno = pd.read_csv('./out_data/posts output folder/gme_filtered_posts_for annotation_2000.csv')


In [3]:
anno.head()

,upvote_ratio,id,num_comments,score,created_et_date,created_et_timestamp,created_et_time,fulltext,is_trade_day,is_trade_time,gme_timeline,positive,negative,neutral,stock
0,0.99,keeu16,0,1,2020-12-16,2020-12-16 13:32:18-05:00,13:32:18,Chart showing the relationship of all 110 Game...,1,1,0,2.0,3.0,7.0,GME
1,1.00,keioza,0,1,2020-12-16,2020-12-16 16:42:30-05:00,16:42:30,"GME short ratio keeps on climbing, short % of ...",1,0,0,9.0,2.0,2.0,GME
2,1.00,khi2vq,0,2,2020-12-21,2020-12-21 09:08:16-05:00,09:08:16,“Having discussions” we know what that means! ...,1,0,0,9.0,1.0,0.0,GME
3,1.00,kkwei5,0,1,2020-12-26,2020-12-26 22:38:31-05:00,22:38:31,First time investor (Canadian) looking to jump...,0,0,0,6.0,3.0,3.0,GME
4,1.00,klsoka,0,3,2020-12-28,2020-12-28 10:39:34-05:00,10:39:34,ran a test on etrade to get a sense of borrow%...,1,1,0,9.0,2.0,1.0,GME


In [7]:
print(anno['positive'].min())
print(anno['positive'].max())
print(anno['negative'].min())
print(anno['negative'].max())
print(anno['neutral'].min())
print(anno['neutral'].max())

-10.0
73.0
-10.0
25.0
-5.0
100.0


In [9]:
anno.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   upvote_ratio          32135 non-null  float64
 1   id                    32135 non-null  object 
 2   num_comments          32135 non-null  int64  
 3   score                 32135 non-null  int64  
 4   created_et_date       32135 non-null  object 
 5   created_et_timestamp  32135 non-null  object 
 6   created_et_time       32135 non-null  object 
 7   fulltext              32135 non-null  object 
 8   is_trade_day          32135 non-null  int64  
 9   is_trade_time         32135 non-null  int64  
 10  gme_timeline          32135 non-null  int64  
 11  positive              32116 non-null  float64
 12  negative              32118 non-null  float64
 13  neutral               32117 non-null  float64
 14  stock                 32135 non-null  object 
dtypes: float64(4), int6

In [11]:
anno['is_trade_day'].value_counts()

is_trade_day
1    23420
0     8715
Name: count, dtype: int64

In [13]:
anno_temp = anno[(anno['is_trade_day'] == 1) & (anno['is_trade_time'] == 1)]
anno_temp.shape

(11173, 15)

In [15]:
anno_temp= anno_temp[anno_temp['fulltext'].str.len() > 15]
anno_temp.shape

(10642, 15)

In [17]:
anno_temp['created_et_date'] = pd.to_datetime(anno_temp['created_et_date'])
anno_temp= anno_temp.sort_values(by="created_et_date", ascending=True)
anno_temp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10642 entries, 0 to 17090
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   upvote_ratio          10642 non-null  float64       
 1   id                    10642 non-null  object        
 2   num_comments          10642 non-null  int64         
 3   score                 10642 non-null  int64         
 4   created_et_date       10642 non-null  datetime64[ns]
 5   created_et_timestamp  10642 non-null  object        
 6   created_et_time       10642 non-null  object        
 7   fulltext              10642 non-null  object        
 8   is_trade_day          10642 non-null  int64         
 9   is_trade_time         10642 non-null  int64         
 10  gme_timeline          10642 non-null  int64         
 11  positive              10636 non-null  float64       
 12  negative              10636 non-null  float64       
 13  neutral              

In [19]:
anno_temp = anno_temp.drop_duplicates(subset=['fulltext'], keep='first')
anno_temp.shape

(10547, 15)

In [21]:
min_sample = anno_temp.groupby('created_et_date', group_keys=False).apply(
    lambda x: x.sample(1, random_state=42)
)

# Step 2: Calculate how many additional rows are needed.
num_min_sample = len(min_sample)
additional_needed = 600 - num_min_sample

# Step 3: Remove the rows already selected to avoid duplicates.
remaining = anno_temp.drop(min_sample.index)

# Step 4: Randomly sample the remaining rows.
additional_sample = remaining.sample(n=additional_needed, random_state=42)

# Step 5: Combine and shuffle the final sample.
final_sample = pd.concat([min_sample, additional_sample]).sample(frac=1, random_state=42).reset_index(drop=True)
final_sample.head(50)

/var/folders/8r/lp807dgx5l1g14nqdf2kk0_m0000gn/T/ipykernel_65363/785148806.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  min_sample = anno_temp.groupby('created_et_date', group_keys=False).apply(


,upvote_ratio,id,num_comments,score,created_et_date,created_et_timestamp,created_et_time,fulltext,is_trade_day,is_trade_time,gme_timeline,positive,negative,neutral,stock
0,1.00,laa9d4,0,1,2021-02-01,2021-02-01 13:30:44-05:00,13:30:44,Best broker to buy GME stocks?,1,1,2,0.0,0.0,10.0,GME
1,1.00,la6xmz,0,1,2021-02-01,2021-02-01 11:19:05-05:00,11:19:05,"$400 GME Holders, we come back for you🚀💎🤲",1,1,2,9.0,0.0,2.0,GME
2,1.00,l6d8j6,0,1,2021-01-27,2021-01-27 15:11:18-05:00,15:11:18,Let’s fucking goooooo thanks mates GME 🦄🦄🦄🦄🦄🦄🦄...,1,1,1,10.0,0.0,0.0,GME
3,0.93,l68ikp,0,35,2021-01-27,2021-01-27 11:41:16-05:00,11:41:16,PSA: The shitload of bots shilling against GME...,1,1,1,9.0,2.0,2.0,GME
4,1.00,l83m9b,0,1,2021-01-29,2021-01-29 15:43:04-05:00,15:43:04,"If they block your GME , block the chain. Snag...",1,1,2,9.0,1.0,0.0,GME
5,1.00,l6cub4,0,1,2021-01-27,2021-01-27 14:55:12-05:00,14:55:12,I'm also a bot that tells you that GME is not ...,1,1,1,6.0,4.0,3.0,GME
6,0.50,kv3s7y,0,0,2021-01-11,2021-01-11 09:33:54-05:00,09:33:54,GME TO THE MOON 🚀 🚀 🚀 F FOR MELVIN.,1,1,0,10.0,0.0,0.0,GME
7,1.00,la8bx8,0,1,2021-02-01,2021-02-01 12:14:08-05:00,12:14:08,GME Volume continually increasing?,1,1,2,9.0,1.0,0.0,GME
8,1.00,laabl0,0,1,2021-02-01,2021-02-01 13:33:06-05:00,13:33:06,Why are WS not just going long on GME?,1,1,2,6.0,4.0,0.0,GME
9,1.00,lab532,1,1,2021-02-01,2021-02-01 14:05:23-05:00,14:05:23,Never seen so many different currencies... Sti...,1,1,2,9.0,2.0,3.0,GME


In [23]:
final_sample1 = final_sample[['id','created_et_date','fulltext','positive','negative','neutral']]

In [29]:
final_sample_anno = final_sample[['id','fulltext']]
final_sample_anno.to_csv('./out_data/posts output folder/600_final_annotation_sample_042025.csv', index = False)

In [25]:
final_sample['created_et_date'].value_counts()

created_et_date
2021-02-01    234
2021-01-27     91
2021-01-28     59
2021-02-02     38
2021-01-29     33
2021-02-04     22
2021-01-25     16
2021-01-26     15
2021-02-25     11
2021-02-05     10
2021-02-03      8
2021-02-09      7
2021-02-26      5
2021-02-18      5
2021-01-22      5
2021-01-15      3
2021-01-14      3
2021-02-11      2
2020-12-23      2
2021-01-19      2
2021-02-22      2
2021-02-12      2
2021-02-17      2
2021-01-20      2
2021-02-08      2
2021-01-21      2
2021-02-24      1
2021-01-13      1
2020-12-29      1
2021-01-06      1
2020-12-30      1
2021-02-19      1
2020-12-28      1
2021-01-12      1
2021-02-10      1
2021-01-04      1
2020-12-16      1
2021-01-08      1
2021-02-23      1
2020-12-31      1
2020-12-24      1
2021-01-11      1
2021-02-16      1
Name: count, dtype: int64

In [40]:
#taking 20 for practice 
filtered_anno_temp = anno_temp[~anno_temp['id'].isin(final_sample['id'])]
filtered_anno_temp_unique_dates = filtered_anno_temp.drop_duplicates(subset='created_et_date')

# Randomly sample 20 rows from the unique dates
random_sample = filtered_anno_temp_unique_dates.sample(n=30, random_state=42)
random_sample[['id','created_et_date','fulltext']].to_csv('./out_data/posts output folder/practice_annotation_questions.csv', index = False)
